In [1]:
# Import our dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import load_model
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

In [ ]:
# Loading the dataset from the resources folder

charity_df = pd.read_csv(Path('Resources/charity_data.csv'))

In [ ]:
charity_df

# Exploratory Data Analysis (EDA):

1. Drop the unnamed column as it does not contribute to clustering or add value to input data
2. List the DataFrame's data types to ensure they're aligned to the type of data stored on each column.
3. Is there any column whose data type need to be changed? If so, make the corresponding adjustments.
4. Is there any unnecessary column that needs to be dropped? If so, make the corresponding adjustments.
5. Check for duplicates.
6. In order to use unsupervised learning algorithms, all the features should be numeric, and also, on similar scales.
7. Rename the column if needed.
8. Check for categorical data in columns.
9. Determine the number of unique values for each column.
10. For those columns that have more than 10 unique values, determine the number of data points for each unique value.
11. Use the number of data points for each unique value to pick a cutoff point to bin "rare" categorical variables together in a new value, Other, and then check if the binning was successful.
12. What variable(s) are considered the target(s) for your model?
13. What variable(s) are considered the feature(s) for your model?

In [ ]:
charity_df.shape

In [ ]:
charity_df.dtypes

In [ ]:
# Find null values
for column in charity_df.columns:
    print(f"Column {column} has {charity_df[column].isnull().sum()} null values")


In [ ]:
# Find duplicate entries
print(f"Duplicate entries: {charity_df.duplicated().sum()}")

In [ ]:
# A list of the columns from the original DataFrame
charity_df.columns

In [ ]:
charity_df.columns

In [ ]:
charity_df.info()

In [ ]:
# Function checking for missing values
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

In [ ]:
missing_values_table(charity_df)

## Data Preprocessing:

In [ ]:
# Drop 'EIN', 'NAME' columns from the dataframe

charity_df = charity_df.drop(['EIN', 'NAME'], axis=1)
charity_df

In [ ]:
charity_df["INCOME_AMT"]

In [ ]:
# new data frame with split value columns
charity_df[['INCOME_LOWER','INCOME_UPPER']] = charity_df["INCOME_AMT"].str.split("-", n = 1, expand = True)
charity_df

In [ ]:
charity_df = charity_df.fillna(0)
charity_df

In [ ]:
charity_df['INCOME_UPPER'] = charity_df['INCOME_UPPER'].replace('M','', regex=True)
charity_df['INCOME_LOWER'] = charity_df['INCOME_LOWER'].replace('M','', regex=True)

In [ ]:
charity_df["INCOME_LOWER"] 

In [ ]:
# Generate our categorical variable lists
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning

charity_df["APPLICATION_TYPE"].value_counts() 

In [ ]:
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T14', 'T25', 'T29', 'T15', 'T17']

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T14', 'T25', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    charity_df['APPLICATION_TYPE'] = charity_df['APPLICATION_TYPE'].replace(app,"Other")
    
# Check to make sure binning was successful
charity_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning

my_value_count = charity_df["CLASSIFICATION"].value_counts()
# a_list[0]
my_value_count.index
my_value_count

In [ ]:
# Getting the values that need to be binned in 'classifications_to_replace'
my_value_count = my_value_count.loc[~my_value_count.index.isin(['C1000', 'C2000', 'C1200', 'C3000', 'C2100'])]
my_value_count.index
classifications_to_replace = my_value_count.index.tolist()
classifications_to_replace

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Replace in dataframe
for cls in classifications_to_replace:
    charity_df['CLASSIFICATION'] = charity_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
charity_df['CLASSIFICATION'].value_counts()

In [ ]:
# Drop 'EIN', 'NAME' columns from the dataframe

charity_df = charity_df.drop(['INCOME_AMT'], axis=1)
charity_df

In [ ]:
# Save as a csv to check the values of 'others' category
# Note to avoid any issues later, use encoding="utf-8"
charity_df.to_csv("check_df.csv", encoding="utf-8", index=False)

In [ ]:
# Generate our categorical variable lists
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

In [ ]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

In [ ]:
# charity_df[charity_cat].info()
# charity_df[charity_cat]
charity_df

In [ ]:
charity_df.dtypes

In [ ]:
charity_df = charity_df.reset_index()

In [ ]:
# Drop the class column
X_charity = charity_df.drop(["IS_SUCCESSFUL"], axis='columns')
X_charity.head()


In [ ]:
y = charity_df["IS_SUCCESSFUL"]
y

In [ ]:
# Transforming y_moons to a vertical vector
# y_charity = y_charity.reshape(-1, 1)

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
X_dummies = pd.get_dummies(X_charity)
X = X_dummies
print (X.columns)
X

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model:

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

In [ ]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

In [ ]:
# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

In [ ]:
# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [ ]:
# Check the structure of the Sequential model
nn_model.summary()

In [ ]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

In [ ]:
print(tf.__version__)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Create a DataFrame containing training history
history_df = pd.DataFrame(fit_model.history, index=range(1,len(fit_model.history["loss"])+1))

# Plot the accuracy
history_df.plot(y="accuracy")

In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE